In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
from generative_social_choice.datasets.datasets import get_dataset
from generative_social_choice.objects.agents import MultiLevelAgent
from generative_social_choice.objects.comparers import BasicComparer
import pandas as pd

## Set up some rudimentary agents

In [95]:
dataset = get_dataset("prolific")
df = dataset.load()
agent_ids = dataset.get_agent_ids()["experiment1"]

agent_ids_to_comment = {
    agent_id: dataset.get_transcript_from_agent_id(agent_id)[
        dataset.get_transcript_from_agent_id(agent_id)["step"] == 7
    ]["text"].values[0]
    for agent_id in agent_ids
}

approval_level = 5

agents = [
    MultiLevelAgent(
        id=agent_id,
        prompt_type="basic",
        approval_level=approval_level,
        agent_opinion=comment,
    )
    for agent_id, comment in agent_ids_to_comment.items()
]

## Set up comparer agent

Currently only a basic prompt (no CoT, no fewshot) is implemented, but we can easily add more.

In [96]:
comparer = BasicComparer(id=0, prompt_type="basic")

## Make single comparer query

Here, `center_agent` and `agent0` both are pro-personalization, and `agent1` is against, so we expect the output of the query to be 0 (the index of the closest agent).

In [97]:
center_agent = agents[0]
center_agent.get_description()

'I believe that the capacity of chatbots to be personalized will allow them to remember our preferences for information.  At the very least, it should remember our previous dialogs, if only to prevent needless repetition.  I believe we should be able to control the level of personalization, much as we set security preferences in other online actions.'

In [98]:
agent0 = agents[2]
agent0.get_description()

'I believe that a chatbot should be able to personalize responses to your questions or needs based on past conversations, information voluntarily provided or even by means of a profile you setup. If you had your own user account that you opted into, then it now becomes a choice wether or not the chatbot service uses a personalized approach. '

In [99]:
agent1 = agents[7]
agent1.get_description()

'I believe that chatbots should not be personalized. I would prefer to receive generic responses based on data and not have the chatbot try to learn my behaviors or personality. I feel that this could hinder the information given by the chatbot if it is too focused on trying to tailor information to you rather than just providing the information requested. '

In [100]:
closest_agent_idx, log_dict = comparer.get_closest_agent_idx(
    center_agent=center_agent, other_agents=[agent0, agent1]
)

In [101]:
closest_agent_idx

0

### sidenote: comparer in theory supports many agents

In theory you can give it `center_agent`, and a list of arbitrarily many agents, and it returns the single closest agent. However the prompt is written with 2 in mind, so it might give worse results. (Specifically, the prompt gives the two output examples 0 if Person 0 is closest, 1 if Person 1 is closest, so possibly the LLM is biased towards putting 0 and 1.)

In [102]:
agent2 = agents[10]
agent2.get_description()

'I certainly think that someone who uses a chatbot for everyday things should definitely be personalized. It would make solutions to your problems more tailored to you. Alot of people have common thoughts on things but everyone is different in every way such as their mindset.'

In [103]:
closest_agent_idx, log_dict = comparer.get_closest_agent_idx(
    center_agent=center_agent, other_agents=[agent0, agent1, agent2]
)

In [104]:
closest_agent_idx

0

## computing nearest neighbors

To compute the `m` nearest neighbors of some `center_agent` from a list of agents `agents`, run the following

In [105]:
center_agent = agents[7]
center_agent.get_description()

'I believe that chatbots should not be personalized. I would prefer to receive generic responses based on data and not have the chatbot try to learn my behaviors or personality. I feel that this could hinder the information given by the chatbot if it is too focused on trying to tailor information to you rather than just providing the information requested. '

In [106]:
other_agents = list(set(agents).difference({center_agent}))

In [107]:
m_closest, logs = comparer.find_nearest_neighbors(
    center_agent=center_agent, agents=other_agents, m=5
)

In [108]:
print("center agent:")
print(center_agent.id)
print(center_agent.get_description())
print("++++++++++++++++++++++")

for agent in m_closest:
    print(agent.id)
    print(agent.get_description())
    print("+++++++++++++")

center agent:
650950e1ab264bd2967f7f9d
I believe that chatbots should not be personalized. I would prefer to receive generic responses based on data and not have the chatbot try to learn my behaviors or personality. I feel that this could hinder the information given by the chatbot if it is too focused on trying to tailor information to you rather than just providing the information requested. 
++++++++++++++++++++++
63d87d4ba0b86684507f201d
Chatbots must provide relevant information to users. If they do not then they would not be a useful. And in some cases their suggestions would be dangerous so this is a must.
+++++++++++++
60a877548e684d43854de84d
I think it is important because it makes the user feel more personable and that it's actually listening. It cares about the person
+++++++++++++
64a80dd34fd0539ee3761e6a
I believe that chatbots such as ChatGPT should be personalized to a significant extent, but within clear ethical boundaries. Personalization is important because it enhan